In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, MultiHeadAttention, Dense, Input, GlobalAveragePooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

file = open('Requirements.txt', 'r')

lines = []
for i in file:
    lines.append(i)

data = ""
for i in lines:
    data = ' '.join(lines)

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')

data = data.split()
data = ' '.join(data)

#remove all special characters and numbers and punctuations
data = ''.join([i for i in data if not i.isdigit()])
data = ''.join([i for i in data if i.isalpha() or i == ' '])

data = data.lower()



tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(10, len(sequence_data)):
    words = sequence_data[i-10:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)

X = []
y = []

for i in sequences:
    X.append(i[0:10])
    y.append(i[10])

X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)

input_layer = Input(shape=(10,))
embedding = Embedding(vocab_size, 128)(input_layer)
transformer_block = MultiHeadAttention(num_heads=8, key_dim=128)(embedding, embedding)
x = GlobalAveragePooling1D()(transformer_block)
x = Dropout(0.1)(x)
x = Dense(1000, activation="relu")(x)
output_layer = Dense(vocab_size, activation="softmax")(x)

model = Model(input_layer, output_layer)

model.summary()

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words_transformer.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])



77
The Length of sequences are:  263
Data:  [[ 1  2  8  6 14  9  1 11  2 47]
 [ 2  8  6 14  9  1 11  2 47 48]
 [ 8  6 14  9  1 11  2 47 48  2]
 [ 6 14  9  1 11  2 47 48  2  4]
 [14  9  1 11  2 47 48  2  4  6]
 [ 9  1 11  2 47 48  2  4  6 14]
 [ 1 11  2 47 48  2  4  6 14  9]
 [11  2 47 48  2  4  6 14  9 11]
 [ 2 47 48  2  4  6 14  9 11  2]
 [47 48  2  4  6 14  9 11  2 49]]
Response:  [48  2  4  6 14  9 11  2 49  1]
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 10, 128)      9856        ['input_2[0][0]']                
                                                                      

In [6]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

model = load_model('next_words_transformer.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""

    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break

    print(predicted_word)
    return predicted_word

while(True):
    text = input("Enter your line: ")

    if text == "0":
        print("Execution completed.....")
        break

    else:
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)

            Predict_Next_Words(model, tokenizer, text)

        except Exception as e:
            print("Error occurred: ", e)


['think', 'and', 'build']
Error occurred:  in user code:

    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    Va